# LAB 6:  Serving baby weight predictions.

**Learning Objectives**

1. Deploy a web application that consumes the machine learning service

## Introduction 
In the pevious notebook, we deployed our model to CAIP. In this notebook, we'll make a [Flask app](https://palletsprojects.com/p/flask/) to show how our models can interact with a web application which could be deployed to [App Engine](https://cloud.google.com/appengine) with the [Flexible Environment](https://cloud.google.com/appengine/docs/flexible).

## Step 1
Let's start with what our users will see. In the `application` folder, we have prebuilt the components for web application. In the templates folder, the <a href="application/templates/index.html">index.html</a> file is the visual GUI our users will make predictions with.

It works by using an HTML [form](https://www.w3schools.com/html/html_forms.asp) to make a [POST request](https://www.w3schools.com/tags/ref_httpmethods.asp) to our server, passing along the values captured by the [input tags](https://www.w3schools.com/html/html_form_input_types.asp).

The page will render a little strangely in the notebook since the notebook environment does not run javascript, nor do we have our server up and running. Let's get to that!

In [1]:
import os
os.environ["BUCKET"] = "your-bucket-id-here"

## Step 2
We can set up our server with python using [Flask](https://flask.palletsprojects.com/en/1.1.x/quickstart/). Below, we've already built out most of the application for you.

The `@app.route()` decorator defines a function to handle web reqests. Let's say our website is `www.example.com`. With how our `@app.route("/")` function is defined, our sever will render our <a href="application/templates/index.html">index.html</a> file when users go to `www.example.com/` (which is the default route for a website).

So, when a user pings our server with `www.example.com/predict`, they would use `@app.route("/predict", methods=["POST"])` to make a prediction. The data that gets sent over the internet isn't a dictionary, but a string like below:

`name1=value1&name2=value2` where `name` corresponds to the `name` on the input tag of our html form, and the value is what the user entered. Thankfully, Flask makes it easy to transform into a dictionary with `request.form.to_dict()`, but we still need to transform the data into a format our model expects. We've done this with the `gender2str` and the `plurality2str` functions.

Ok! With our data all set up, let's hit up our CAIP models. Fill in the **TODO** below to connect our web server to our ML pipeline.

In [2]:
%%writefile application/main.py
import os

from flask import Flask
from flask import render_template
from flask import request
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials


credentials = GoogleCredentials.get_application_default()
api = discovery.build("ml", "v1", credentials=credentials)

app = Flask(__name__)


def get_prediction(features):
    project = "ddetering-experimental"
    model_name = os.getenv("MODEL_NAME", "babyweight")
    version_name = os.getenv("VERSION_NAME", "ml_on_gcp")

    input_data = {"instances": [features]}
    # TODO: Write the code to make a prediction against a CAIP deployed model
    parent = "projects/{0}/models/{1}/versions/{2}".format(
        project, model_name, version_name)
    prediction = api.projects().predict(body=input_data, name=parent).execute()

    return prediction["predictions"][0]["weight"][0]


@app.route("/")
def index():
    return render_template("index.html")


@app.route("/predict", methods=["POST"])
def predict():
    def gender2str(val):
        genders = {"unknown": "Unknown", "male": "True", "female": "False"}
        return genders[val]

    def plurality2str(val):
        pluralities = {"1": "Single(1)", "2": "Twins(2)", "3": "Triplets(3)"}
        if features["is_male"] == "Unknown" and int(val) > 1:
            return "Multiple(2+)"
        return pluralities[val]

    data = request.form.to_dict()
    mandatory_items = ["babyGender",
                       "motherAge",
                       "plurality",
                       "gestationWeeks"]
    for item in mandatory_items:
        if item not in data.keys():
            return "Set all items."

    features = {}
    features["is_male"] = gender2str(data["babyGender"])
    features["mother_age"] = float(data["motherAge"])
    features["plurality"] = plurality2str(data["plurality"])
    features["gestation_weeks"] = float(data["gestationWeeks"])

    prediction = get_prediction(features)

    return "{:.2f} lbs.".format(prediction)


if __name__ == '__main__':
    # This is used when running locally. Gunicorn is used to run the
    # application on Google App Engine. See entrypoint in app.yaml.
    app.run(host='127.0.0.1', port=8080)


Overwriting application/main.py


## Step 3
So how do we know that it works? We'll have to deploy our website and find out! Notebooks aren't made for website deployment, so we'll move our operation to the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true).

By default, the shell doesn't have Flask installed, so copy over the following command to install it.

`python3 -m pip install --user Flask==0.12.1`

Next, we'll need to copy our web app to the Cloud Shell. We can use [GCS](https://cloud.google.com/storage) as an inbetween.

In [3]:
%%bash
gsutil rm -r application/ gs://$BUCKET/baby_app
gsutil cp -r application/ gs://$BUCKET/baby_app

CommandException: "rm" command does not support "file://" URLs. Did you mean to use a gs:// URL?
Copying file://application/main.py [Content-Type=text/x-python]...
Copying file://application/app.yaml [Content-Type=application/octet-stream]...  
Copying file://application/templates/index.html [Content-Type=text/html]...     
Copying file://application/templates/.ipynb_checkpoints/index-checkpoint.html [Content-Type=text/html]...
- [4 files][ 10.7 KiB/ 10.7 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://application/static/baby.png [Content-Type=image/png]...
Copying file://application/.ipynb_checkpoints/main-checkpoint.py [Content-Type=text/x-python]...
Copying file://application/.ipynb_checkpoints/app-checkpoint.yaml [

Run the below cell, and copy the output into the [Google Cloud Shell](https://console.cloud.google.com/home/dashboard?cloudshell=true)

In [4]:
%%bash
echo rm -r baby_app/
echo mkdir baby_app/
echo gsutil cp -r gs://$BUCKET/baby_app ./
echo python3 baby_app/main.py

rm -r baby_app/
mkdir baby_app/
gsutil cp -r gs://ddetering-experimental/baby_app ./
python3 baby_app/main.py


## Step 4
Time to play with the website! The cloud shell should now say something like `* Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)`. Click on the `http` link to go to your shiny new website. Fill out the form and give it a minute or two to process its first prediction. After the first one, the rest of the predictions will be lightning fast.

Did you get a prediction? If not, the Google Cloud Shell will spit out a stack trace of the error to help narrow it down. If yes, congratulations! Great job on bringing all of your work together for the users.